In [ ]:
#file_path for the weights of the trained model to be used
#https://polybox.ethz.ch/index.php/s/ibQHQSyQsqBKjCl

In [ ]:
#cells where file paths need to be altered to rerun 
#Loc-1: Read train.txt
#Loc-2: Split into training and validation sets and save as txt
#Loc-3: File paths for training,validation,test,all_images 
#Loc-4: Function to read the pixel data of each image and convert it to target size
#Loc-5: File path for loading trained weights (polybox reference path provided at the top)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [ ]:
pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 38kB/s 
     |████████████████████████████████| 419kB 40.7MB/s 
     |████████████████████████████████| 3.0MB 39.9MB/s 


In [ ]:
#import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random
from PIL import Image
from matplotlib import image
import tensorflow as tf
import cv2
from glob import glob
from keras.layers import Input


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [ ]:
import keras
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.layers import *
from keras.models import Model, load_model
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.compiler import tensorrt as trt
from keras.models import Sequential
import keras.backend as K
import tensorflow as tf
from keras.preprocessing import image

In [ ]:
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from random import choice, sample

In [ ]:
#Loc-1: read training txt
train_path = "drive/My Drive/Task 4/train.txt"
train=pd.read_csv(train_path,header=None)
#Loc-2: split into training and validation sets and save as txt
X_train,X_val=train_test_split(train,test_size=0.25, random_state=1)
np.savetxt("drive/My Drive/Task 4/X_train.txt", X_train, delimiter=",", fmt='%s')
np.savetxt("drive/My Drive/Task 4/X_val.txt", X_val, delimiter=",", fmt='%s')

In [ ]:
#Loc-3: File paths for training, validation,test txts,images path
train_file_path = "drive/My Drive/Task 4/X_train.txt"
val_file_path = "drive/My Drive/Task 4/X_val.txt"
test_file_path="drive/My Drive/Task 4/test.txt"
images_path = "drive/My Drive/Task 4/Food"

In [ ]:
#read training, validation and test txt
train_triplets=pd.read_csv(train_file_path,header=None) 
val_triplets=pd.read_csv(val_file_path,header=None)
test_triplets=pd.read_csv(test_file_path,header=None)

In [ ]:
#reading triplet txts to save anchor, positive and negative image filenames
train_anchor=[]
train_pos=[]
train_neg=[]
for i in range(0,44636):
  x=str(train_triplets.values[i][0])
  img1=x[0:5]
  train_anchor.append(img1)
  img2=x[6:11]
  train_pos.append(img2)
  img3=x[12:17]
  train_neg.append(img3)

In [ ]:
val_anchor=[]
val_pos=[]
val_neg=[]
for i in range(0,14879):
  x=str(val_triplets.values[i][0])
  img1=x[0:5]
  val_anchor.append(img1)
  img2=x[6:11]
  val_pos.append(img2)
  img3=x[12:17]
  val_neg.append(img3)

In [ ]:
test_anchor=[]
test_pos=[]
test_neg=[]
for i in range(0,59544):
  x=str(test_triplets.values[i][0])
  img1=x[0:5]
  test_anchor.append(img1)
  img2=x[6:11]
  test_pos.append(img2)
  img3=x[12:17]
  test_neg.append(img3)

In [ ]:
train_tuples=list(zip(train_anchor,train_pos,train_neg))
val_tuples=list(zip(val_anchor,val_pos,val_neg))
test_tuples=list(zip(test_anchor,test_pos,test_neg))

In [ ]:
#Loc-4:function to read the pixel data of each image and convert it to target size
def read_img(img_x):
    path="drive/My Drive/Task 4/food/"+img_x+'.jpg'
    img = image.load_img(path, target_size=(225,225))
    img = np.array(img).astype(np.float)
    return preprocess_input(img)

In [ ]:
#generating tuples
def gen(list_tuples,batch_size=120):
  while True:
    batch_tuples=sample(list_tuples,batch_size)
    labels=[1]*len(batch_tuples)
    X1=[x[0] for x in batch_tuples]
    X1=np.array([read_img(x) for x in X1])
    X2=[x[1] for x in batch_tuples]
    X2=np.array([read_img(x) for x in X2])
    X3=[x[2] for x in batch_tuples]
    X3=np.array([read_img(x) for x in X3])
    yield [X1,X2,X3],labels

In [ ]:
#defining triplet loss
def triplet_loss(y_true,y_pred,alpha=0.25):
  total_length = y_pred.shape.as_list()[-1]
  anchor=y_pred[:,0:int(total_length*1/3)]
  positive=y_pred[:,int(total_length*1/3):int(total_length*2/3)]
  negative=y_pred[:,int(total_length*2/3):int(total_length*3/3)]
  pos_dist=K.sum(K.square(anchor-positive),axis=1)
  neg_dist=K.sum(K.square(anchor-negative),axis=1)
  basic_loss=pos_dist-neg_dist+alpha
  loss=K.maximum(basic_loss,0.0)
  return loss

In [ ]:
#defining label prediction based on triplet loss
def triplet_loss_label(y_pred):
  total_length = len(y_pred)
  anchor=np.array(y_pred[0:int(total_length*1/3)])
  positive=np.array(y_pred[int(total_length*1/3):int(total_length*2/3)])
  negative=np.array(y_pred[int(total_length*2/3):int(total_length*3/3)])
  pos_dist=np.sum(np.square(anchor-positive))
  neg_dist=np.sum(np.square(anchor-negative))
  basic_loss=pos_dist-neg_dist
  if basic_loss>=0:
    label=0
  else:
    label=1  
  return label

In [ ]:
def base_model(input_vgg):
  vgg=VGG16(input_tensor=input_vgg,weights='imagenet', include_top=False)
  for x in vgg.layers:
    x.trainable=False
  out=vgg.output
  v_x1=Conv2D(96,kernel_size=(6),strides=(6),padding="same")(out)
  v_x2=MaxPool2D()(v_x1)
  v_x3=Dropout(0.6)(v_x2)
  #final=Dense(64,activation='sigmoid')(x3)
  base_model=Model(input_vgg,v_x3)
  return base_model.output

In [ ]:
def baseline_model():
  input_1=Input(shape=(225,225,3))
  input_2=Input(shape=(225,225,3))
  input_3=Input(shape=(225,225,3))
  vgg=VGG16(weights='imagenet', include_top=False)
  for x in vgg.layers:
    x.trainable=False
  base_model=Sequential()
  for x in vgg.layers:
    base_model.add(x)
  #base_model.add(Conv2D(64,kernel_size=(6),strides=(6),padding="same"))
  #base_model.add(Dropout(0.6))
  base_model.add(Dense(64,activation='sigmoid'))
  base_model.add(Dense(32,activation='sigmoid'))
  base_model.add(Dense(16,activation='sigmoid'))
  #base_model.add(Dense(8,activation='sigmoid'))
  x1=base_model(input_1)
  x2=base_model(input_2)
  x3=base_model(input_3)
  f_x1=Concatenate(axis=-1)([GlobalMaxPool2D()(x1),GlobalAvgPool2D()(x1)])
  f_x2=Concatenate(axis=-1)([GlobalMaxPool2D()(x2),GlobalAvgPool2D()(x2)])
  f_x3=Concatenate(axis=-1)([GlobalMaxPool2D()(x3),GlobalAvgPool2D()(x3)])
  concat_vector=concatenate([f_x1,f_x2,f_x3],axis=-1,name='concat')
  model=Model([input_1,input_2,input_3],concat_vector)
  model.compile(loss=triplet_loss,optimizer=Adam(0.0001))
  model.summary()
  return model

In [ ]:
model=baseline_model()

58892288/58889256 [==============================] - 1s 0us/step
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 225, 225, 3)  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 225, 225, 3)  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 225, 225, 3)  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       multiple             14750128    input_1[0][0]                    
                           

In [ ]:
#model.fit_generator(gen(train_tuples,batch_size=100),use_multiprocessing=True,
#                  validation_data=gen(val_tuples,batch_size=50),epochs=100,verbose=1,steps_per_epoch=25,validation_steps=10)
model.fit_generator(gen(train_tuples,batch_size=100),use_multiprocessing=True,
                    epochs=100,verbose=1,steps_per_epoch=25)

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:718: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


25/25 [==============================] - 3804s 152s/step - loss: 0.2492
Epoch 2/100
25/25 [==============================] - 78s 3s/step - loss: 0.2489
Epoch 3/100
25/25 [==============================] - 73s 3s/step - loss: 0.2483
Epoch 4/100
25/25 [==============================] - 72s 3s/step - loss: 0.2468
Epoch 5/100
25/25 [==============================] - 71s 3s/step - loss: 0.2451
Epoch 6/100
25/25 [==============================] - 72s 3s/step - loss: 0.2413
Epoch 7/100
25/25 [==============================] - 71s 3s/step - loss: 0.2357
Epoch 8/100
25/25 [==============================] - 71s 3s/step - loss: 0.2251
Epoch 9/100
25/25 [==============================] - 71s 3s/step - loss: 0.2140
Epoch 10/100
25/25 [==============================] - 70s 3s/step - loss: 0.2021
Epoch 11/100
25/25 [==============================] - 71s 3s/step - loss: 0.1934
Epoch 12/100
25/25 [==============================] - 70s 3s/step - loss: 0.1890
Epoch 13/100
25/25 [=========================

In [ ]:
#file_path for saving weights from each run
#file_path="drive/My Drive/Task 4/triplet.h5"
#model.save_weights(file_path)
#
#Loc-5:file path for loading trained weights (polybox reference path provided at the top)
file_path="drive/My Drive/Task 4/triplet_trained.h5"
model.load_weights(file_path)

In [ ]:
#making predictions for test triplets
predictions=list()
for i in range(0,59544):
  X1=np.array([read_img(test_anchor[i])])
  X2=np.array([read_img(test_pos[i])])
  X3=np.array([read_img(test_neg[i])])
  dist=model.predict([X1,X2,X3]).ravel().tolist()
  label=triplet_loss_label(dist)
  predictions.append(label)

In [ ]:
np.savetxt("pred_trained_weights.csv", predictions, delimiter=",", fmt='%s')